In [1]:
# IMporting all the necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from collections import deque
import random
import tqdm as tqdm

In [4]:
!pip install mlcompute


ERROR: Could not find a version that satisfies the requirement mlcompute (from versions: none)
ERROR: No matching distribution found for mlcompute


In [3]:
# enabling m1 gpus
from tensorflow.python.compiler.mlcompute import mlcompute; 
mlcompute.set_mlc_device(device_name='gpu')

ModuleNotFoundError: No module named 'tensorflow.python.compiler.mlcompute'

In [2]:
# Define the DQN agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)  # Replay memory
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self.build_model()

    # Build the deep Q-network
    def build_model(self):
        model = tf.keras.Sequential()
        model.add(layers.Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(layers.Dense(24, activation='relu'))
        model.add(layers.Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=self.learning_rate))
        return model

    # Store experience in replay memory
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    # Choose action using epsilon-greedy policy
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    # Update the Q-network using experience replay
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
# Define the checkers environment
class CheckersEnvironment:
    def __init__(self):
        self.state_size = 8  # Size of the checkers board
        self.action_size = 4  # Number of possible actions
        # Define other environment-specific variables and methods here

    def reset(self):
        # Reset the environment to the initial state
        pass

    def step(self, action):
        # Execute the given action in the environment and return the next state, reward, and done flag
        pass

In [ ]:
# Training the DQN agent
def train_dqn_agent():
    # Initialize the checkers environment and DQN agent
    env = CheckersEnvironment()
    agent = DQNAgent(env.state_size, env.action_size)

    num_episodes = 1000  # Number of training episodes
    batch_size = 32  # Size of the minibatch for experience replay

    for episode in range(num_episodes):
        state = env.reset()
        state = np.reshape(state, [1, env.state_size])
        done = False
        total_reward = 0

        while not done:
            # Agent takes an action
            action = agent.act(state)

            # Environment processes the action
            next_state, reward, done = env.step(action)
            next_state = np.reshape(next_state, [1, env.state_size])

            # Store the experience in agent's memory
            agent.remember(state, action, reward, next_state, done)

            # Update the current state
            state = next_state
            total_reward += reward

            if done:
                print("Episode:", episode, "Total Reward:", total_reward)

            if len(agent.memory) > batch_size:
                # Train the agent by replaying experiences from the memory
                agent.replay(batch_size)

In [ ]:
# Play a game using a trained DQN agent
def play_game_with_dqn_agent():
    # Initialize the checkers environment and DQN agent
    env = CheckersEnvironment()
    agent = DQNAgent(env.state_size, env.action_size)

    # Load the trained weights into the DQN model
    agent.model.load_weights('dqn_weights.h5')

    state = env.reset()
    state = np.reshape(state, [1, env.state_size])
    done = False

    while not done:
        # Agent selects the best action
        action = agent.act(state)

        # Environment processes the action
        next_state, reward, done = env.step(action)
        next_state = np.reshape(next_state, [1, env.state_size])

        # Update the current state
        state = next_state

        # Render the checkers board and perform other necessary steps for visualization

In [ ]:
# Train the DQN agent
train_dqn_agent()

# Save the trained weights
agent.model.save_weights('dqn_weights.h5')

# Play a game using the trained DQN agent
play_game_with_dqn_agent()

In [ ]:
def render_board(board):
    # Print the board with ASCII characters
    print("   A B C D E F G H")
    print("   ----------------")
    for row in range(len(board)):
        print(row + 1, "|", end=" ")
        for col in range(len(board[row])):
            if board[row][col] == 0:
                print("-", end=" ")
            elif board[row][col] == 1:
                print("O", end=" ")  # Player 1's piece
            elif board[row][col] == 2:
                print("X", end=" ")  # Player 2's piece
        print()
    print()

# Example usage:
board = [
    [0, 1, 0, 1, 0, 1, 0, 1],
    [1, 0, 1, 0, 1, 0, 1, 0],
    [0, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [2, 0, 2, 0, 2, 0, 2, 0],
    [0, 2, 0, 2, 0, 2, 0, 2],
    [2, 0, 2, 0, 2, 0, 2, 0]
]

render_board(board)
